In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [2]:
x = torch.tensor([.5, 0.0, 1.0])
y = torch.sigmoid(x)
print(y)

tensor([0.6225, 0.5000, 0.7311])


In [3]:
def pad_to_length(tensor_list, max_len=7):
    padded = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    if padded.size(1) < max_len:
        pad_size = max_len - padded.size(1)
        padded = F.pad(padded, (0, pad_size), value=0)
    else:
        padded = padded[:, :max_len]
    return padded

# Function to process and pad .npy dataset
def process_round_data(npy_file, max_len=7):
    data = np.load(npy_file)
    x = data[:, :-1]
    y = data[:, -1]
    x_tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in x]
    x_padded = pad_to_length(x_tensor_list, max_len=max_len)
    y_tensor = torch.tensor(y, dtype=torch.float)
    return x_padded, y_tensor

# === Load and process each round ===
x_first, y_first = process_round_data('data_for_first_round.npy', max_len=7)
x_second, y_second = process_round_data('data_for_second_round.npy', max_len=7)
x_third, y_third = process_round_data('data_for_third_round.npy', max_len=7)

# === Concatenate all rounds ===
x_all = torch.cat([x_first, x_second, x_third], dim=0)
y_all = torch.cat([y_first, y_second, y_third], dim=0)

# === Final dataset and dataloader ===
dataset_all = TensorDataset(x_all, y_all)
dataloader_all = DataLoader(dataset_all, batch_size=32, shuffle=True)

# === Inspect sizes ===
print("First round input shape:", x_first.shape)
print("First round label shape:", y_first.shape)

print("Second round input shape:", x_second.shape)
print("Second round label shape:", y_second.shape)

print("Third round input shape:", x_third.shape)
print("Third round label shape:", y_third.shape)

print("Combined input shape:", x_all.shape)
print("Combined label shape:", y_all.shape)

First round input shape: torch.Size([2245, 7])
First round label shape: torch.Size([2245])
Second round input shape: torch.Size([74989, 7])
Second round label shape: torch.Size([74989])
Third round input shape: torch.Size([748424, 7])
Third round label shape: torch.Size([748424])
Combined input shape: torch.Size([825658, 7])
Combined label shape: torch.Size([825658])


## MODEL

In [4]:
class EmbeddingNetLinear(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetLinear, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.fc1 = nn.Linear(num_cards * embed_dim, 64)  # Increased capacity
        self.fc2 = nn.Linear(64, 32)                     # New layer
        #self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)                       # Output layer

    def forward(self, x):
        embedded = self.embedding(x)  # Shape: [batch_size, num_cards, embed_dim]
        flat = embedded.view(x.size(0), self.num_cards * self.embed_dim)
        out = F.relu(self.fc1(flat))
        out = F.relu(self.fc2(out))
        #out = F.relu(self.fc3(out))
        return torch.sigmoid(self.fc4(out)).squeeze(1)
    

class EmbeddingNetConv1D(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetConv1D, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=16, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4)

        self.pool = nn.AdaptiveMaxPool1d(1)  # Reduces to [batch, 64, 1]

        self.fc = nn.Linear(64, 1)  # Final output layer

    def forward(self, x):
        x = self.embedding(x)           # [batch, 7, embed_dim]
        x = x.transpose(1, 2)           # [batch, embed_dim, 7]
        x = F.relu(self.conv1(x))       # [batch, 32, 6]
        x = F.relu(self.conv2(x))       # [batch, 64, 4]
        x = F.relu(self.conv3(x))       # [batch, 64, 1]
        x = self.pool(x).squeeze(-1)    # [batch, 64]
        return torch.sigmoid(self.fc(x)).squeeze(1)

## TRAIN FUNCTION

In [5]:
def train_model(model, dataloader, epochs=10, lr=0.001):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # Because we're using sigmoid in the model

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for x_batch, y_batch in dataloader:
            x_batch = x_batch
            y_batch = y_batch

            optimizer.zero_grad()
            preds = model(x_batch)

            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = (preds > 0.5).float()
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        acc = correct / total
        print(f"Epoch {epoch}/{epochs} | Loss: {total_loss:.4f} | Accuracy: {acc:.4f}")

In [8]:
#convModel = EmbeddingNetConv1D(7, 16) 

#train_model(convModel, dataloader_all, 100, 0.0001)
## simple linear 85%
linearModel = EmbeddingNetLinear(7, 16)

train_model(linearModel, dataloader_all, 100, 0.00001)

Epoch 1/100 | Loss: 16962.4100 | Accuracy: 0.6268
Epoch 2/100 | Loss: 16732.2004 | Accuracy: 0.6369
Epoch 3/100 | Loss: 16624.9999 | Accuracy: 0.6374
Epoch 4/100 | Loss: 16526.6715 | Accuracy: 0.6406
Epoch 5/100 | Loss: 16436.4240 | Accuracy: 0.6456
Epoch 6/100 | Loss: 16350.7676 | Accuracy: 0.6506
Epoch 7/100 | Loss: 16266.2956 | Accuracy: 0.6549
Epoch 8/100 | Loss: 16179.7527 | Accuracy: 0.6587
Epoch 9/100 | Loss: 16092.0559 | Accuracy: 0.6619
Epoch 10/100 | Loss: 16005.4049 | Accuracy: 0.6648
Epoch 11/100 | Loss: 15922.2960 | Accuracy: 0.6674
Epoch 12/100 | Loss: 15844.8415 | Accuracy: 0.6696
Epoch 13/100 | Loss: 15772.6453 | Accuracy: 0.6718
Epoch 14/100 | Loss: 15705.0260 | Accuracy: 0.6737
Epoch 15/100 | Loss: 15640.9381 | Accuracy: 0.6756
Epoch 16/100 | Loss: 15578.8517 | Accuracy: 0.6775
Epoch 17/100 | Loss: 15519.0133 | Accuracy: 0.6794
Epoch 18/100 | Loss: 15459.6826 | Accuracy: 0.6813
Epoch 19/100 | Loss: 15402.2772 | Accuracy: 0.6834
Epoch 20/100 | Loss: 15346.2759 | Accura

KeyboardInterrupt: 

## TESTING MODEL

In [29]:
import random
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))      

In [34]:
def get_model_input_based_on_round(cards, round):
    if round == 0:
        input_cards = cards[:2]
    elif round == 1:
        input_cards = cards[:5]
    else:
        input_cards = cards[:7]
    
    # Convert to tensor
    input_tensor = torch.tensor(input_cards, dtype=torch.long)

    # Pad with zeros on the right if needed
    if len(input_tensor) < 7:
        pad_size = 7 - len(input_tensor)
        input_tensor = F.pad(input_tensor, (0, pad_size), value=0)
    else:
        input_tensor = input_tensor[:7]  # Truncate just in case

    return input_tensor.unsqueeze(0)  # shape: (max_len,)
    
    

In [88]:
import ultimate
def test_model_with_games(model, num_games = 100):
    model.eval()
    budget = 0
    allBet = 0
    folds = 0
    flops = 0
    rivers = 0
    preflops = 0

    for i in range(num_games): 
        # generate game (9 cards, played, river, dealer)
        cards = random.sample(num_deck, 9)
        winnings = 0

        # check what round we are in
        round = 0
        round_when_bet = None
        # play  game until it ends
        while True:
            model_input = get_model_input_based_on_round(cards, round)
            
            # Forward pass to get prediction (probability of betting 1)
            with torch.no_grad():
                pred = model(model_input)  # shape: [1, 1]

            pred_prob = pred.item()  # get scalar probability

            # if prediction > 0.5 => bet, else don't bet
            if pred_prob > 0.5 and round_when_bet is None:
                round_when_bet = round
                break

            round += 1

            if round == 3:
                break

        # calculate winnings
        player_hand = [enumerated_deck[card] for card in cards[0:7]]
        dealer_hand = [enumerated_deck[card] for card in cards[2:]]

        player_combination = ultimate.get_best_hand(player_hand)
        dealer_combination = ultimate.get_best_hand(dealer_hand)

        player_rank = winning_hand_ranks[player_combination]
        dealer_rank = winning_hand_ranks[dealer_combination]

        victor = 0 # 0 = dealer, 1 = player
        
        if player_rank > dealer_rank:
            victor = 1
        elif player_rank == dealer_rank:
            result = ultimate.decider(player_combination, player_hand, 
                                    dealer_combination, dealer_hand)
            if result == "player":
                victor = 1
            elif result == "dealer":	
                victor = 0
            else:
                victor = 2	# need to decide about this
                winnings = 0
        else:
            victor = 0

        # check if ante is valid
        dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
        blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

        # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
        if round_when_bet == 0:
            if victor == 1:
                winnings =  4 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -6
            allBet += 6
            preflops += 1
        elif round_when_bet == 1:
            if victor == 1:
                winnings =  2 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -4
            allBet += 4
            flops += 1
        elif round_when_bet == 2:
            if victor == 1:
                winnings =  1 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -3
            allBet += 3
            rivers += 1
        elif round_when_bet == None:
            winnings = -2
            allBet += 2
            folds += 1
        
 
        budget += winnings
        #print("Winnings: ", winnings)
    
    print("Budget is: ", budget)
    print("Betted: ", allBet)
    print("PreFlops: ", preflops)
    print("Flops:", flops)
    print("Rivers:", rivers)
    print("Folds: ", folds)

In [98]:
test_model_with_games(embeddingModel, 10000)

Budget is:  -4951.0
Betted:  36456
PreFlops:  3144
Flops: 1604
Rivers: 672
Folds:  4580
